In [2]:
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize

In [2]:
#from wordcloud import WordCloud
#from textblob import TextBlob

In [3]:
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [24]:
twitter_data = pd.read_csv('clean_twitter_dataset.csv') 
twitter_data.head()

,name,retweet_count,text,created_at,favourite_count,hashtags
0,Pomp,0,wild,2021-03-08,61,[]
1,Pomp,125,rt if you had invested your 1200 stimulus che...,2021-03-08,0,[]
2,Pomp,3,how is your paper trading account going,2021-03-08,86,[]
3,Pomp,132,update gamestop closed today at 194 per share ...,2021-03-08,1906,[]
4,Pomp,6,rt daily letter this morning is pretty spot ...,2021-03-08,0,[]


In [21]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24341 entries, 0 to 24340
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             24341 non-null  object
 1   retweet_count    24341 non-null  int64 
 2   text             24237 non-null  object
 3   created_at       24341 non-null  object
 4   favourite_count  24341 non-null  int64 
 5   hashtags         24341 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


Los tweets contienen información numérica que podría ser de utilidad, sin embargo para el mvp será descartada y nos centraremos unicamente en el sentimiento de los mensajes.

## Erase stopwords

1. Split in words
2. Keep only alphabetic characters
3. apply stopworsa

In [22]:
def splitWords(tweet):
    
    text = str(tweet)
    words = text.split()
    words_no_punc = []
    
    for w in words:
        if w.isalpha():
            words_no_punc.append(w.lower())      
        
    return words_no_punc

In [25]:
twitter_data['sentiment_a'] = twitter_data['text'].apply(lambda x: splitWords(x))

In [28]:
from nltk.corpus import stopwords

In [31]:
def stopWords(words):
    stop_en = stopwords.words('english')
    return stop_en

In [32]:
twitter_data['stopwords'] = twitter_data['text'].apply(lambda x: stopWords(x))

In [38]:
twitter_data['stopwords'][0]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### column clean words

In [45]:
def cleanWords(words, stopwords):
    clean_words = []

    if words not in stopwords:
        clean_words.append(words)
            
    return clean_words

In [46]:
twitter_data['clean_words'] = twitter_data['text'].apply(lambda x: cleanWords(str(x),twitter_data['stopwords'][0]))

In [49]:
twitter_data.clean_words

0                                                 [ wild ]
1        [rt  if you had invested your 1200 stimulus ch...
2               [ how is your paper trading account going]
3        [update gamestop closed today at 194 per share...
4        [rt   daily letter this morning is pretty spot...
                               ...                        
24336       [rt  update etc trading volume within 24hour ]
24337    [ no matter how good of a trader you are 100 w...
24338    [etc is hot now preparing for upmove long time...
24339    [ if everybody gets dumped on why does everyon...
24340                  [ dreams great messengers they are]
Name: clean_words, Length: 24341, dtype: object

In [50]:
# Strip retweet:

def stripRetweet(tweet):
    pattern = r"rt\s+"
    no_mentions = re.sub(pattern,"",tweet)
    return no_mentions

In [56]:
twitter_data['clean_words'] = twitter_data['clean_words'].apply(lambda x: stripRetweet(str(x)))

In [57]:
twitter_data

,name,retweet_count,text,created_at,favourite_count,hashtags,sentiment_a,stopwords,clean_words
0,Pomp,0,wild,2021-03-08,61,[],[wild],"[i, me, my, myself, we, our, ours, ourselves, ...",[' wild ']
1,Pomp,125,rt if you had invested your 1200 stimulus che...,2021-03-08,0,[],"[rt, if, you, had, invested, your, stimulus, c...","[i, me, my, myself, we, our, ours, ourselves, ...",['if you had invested your 1200 stimulus check...
2,Pomp,3,how is your paper trading account going,2021-03-08,86,[],"[how, is, your, paper, trading, account, going]","[i, me, my, myself, we, our, ours, ourselves, ...",[' how is your paper trading account going']
3,Pomp,132,update gamestop closed today at 194 per share ...,2021-03-08,1906,[],"[update, gamestop, closed, today, at, per, sha...","[i, me, my, myself, we, our, ours, ourselves, ...",['update gamestop closed today at 194 per shar...
4,Pomp,6,rt daily letter this morning is pretty spot ...,2021-03-08,0,[],"[rt, daily, letter, this, morning, is, pretty,...","[i, me, my, myself, we, our, ours, ourselves, ...",['daily letter this morning is pretty spot on ...
...,...,...,...,...,...,...,...,...,...
24336,CryptoYoda,21,rt update etc trading volume within 24hour,2017-05-29,0,[],"[rt, update, etc, trading, volume, within]","[i, me, my, myself, we, our, ours, ourselves, ...",['update etc trading volume within 24hour ']
24337,CryptoYoda,0,no matter how good of a trader you are 100 wi...,2017-05-29,17,[],"[no, matter, how, good, of, a, trader, you, ar...","[i, me, my, myself, we, our, ours, ourselves, ...",[' no matter how good of a trader you are 100 ...
24338,CryptoYoda,76,etc is hot now preparing for upmove long time ...,2017-05-29,184,[],"[etc, is, hot, now, preparing, for, upmove, lo...","[i, me, my, myself, we, our, ours, ourselves, ...",['etc is hot now preparing for upmove long tim...
24339,CryptoYoda,0,if everybody gets dumped on why does everyone...,2017-05-29,6,[],"[if, everybody, gets, dumped, on, why, does, e...","[i, me, my, myself, we, our, ours, ourselves, ...",[' if everybody gets dumped on why does everyo...


In [5]:
from textblob import TextBlob

In [6]:
def sentimentAnalysis(clean_words):
    
    variable = TextBlob(clean_words).sentiment
    
    return variable.polarity

In [7]:
twitter_data['polarity'] = twitter_data['clean_words'].apply(lambda x: sentimentAnalysis(str(x)))

## Sentiment column

In [8]:
def sentimentClassification(number):

    category = ""
    if 1>= number > 0:
        category = 'positive'
    elif 0 == number:
        category = 'neutral'
    elif 0 > number >= -1:
        category = 'negative'
        
    return category

In [10]:
twitter_data['sentiment'] = twitter_data['polarity'].apply(lambda x: sentimentClassification(x))

In [12]:
twitter_data.pivot_table(index=['sentiment'], aggfunc='size')

sentiment
negative     3422
neutral      9939
positive    10980
dtype: int64

## Save Clean Dataset

In [13]:
# Export comma-separated variable file
twitter_data = twitter_data.to_csv('clean_tweets.csv', index=False)